In [1]:
#Import libraries
import pandas as pd

In [2]:
#!pip install TextBlob

In [3]:
dirPath = r"\data"
reviewPath = r"reviews_Tools_and_Home_Improvement_5.json.gz"
#reviewPath = r"reviews_Musical_Instruments_5.json.gz"
reviewPath = r"cleanData_reviews_Musical_Instruments_5.json.gz"
path = dirPath + "\\" + reviewPath
path = "preprocessed_reviews_Musical_Instruments_5.pkl.gz"
path = "preprocessed_reviews_Tools_and_Home_Improvement_5.pkl.gz"

In [4]:
data = pd.read_pickle(path)
data.shape

(123707, 7)

In [5]:
negativeReviews = data.loc[data['trueRating'] == 0, :]
negativeReviews.shape[0]


10105

In [6]:
#data = data.iloc[0:size*5, :]

In [7]:
negativeReviews.tail(2)

,masterIndex,asin,reviewText,helpful,overall,trueRating,CleanedJoin
123477,134239,B00JPBDL9W,I have converted three-quarters of my home to ...,"[0, 0]",2.0,0.0,i have convert three quarter of my home to lea...
123556,134322,B00JXSASK2,Amazing that they designed this thing without ...,"[2, 2]",1.0,0.0,amaze that they design this thing without real...


In [8]:
#Pad more negative reviews for better training purposes
testSample = pd.concat([data, negativeReviews])
#testSample = data
#testSample.reset_index(drop=True, inplace=True)
testSample.tail(1)

,masterIndex,asin,reviewText,helpful,overall,trueRating,CleanedJoin
123556,134322,B00JXSASK2,Amazing that they designed this thing without ...,"[2, 2]",1.0,0.0,amaze that they design this thing without real...


In [9]:
import itertools

largeText = testSample.CleanedJoin.to_list()
#largeText = reviewTextProcessing.reviewText.to_list() # Not too good accuracy around 116 missed with %86.94

len(largeText)

133812

In [10]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [11]:
num_words = 15000

In [12]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(largeText)
#tokenizer.word_index

In [13]:
X = largeText
y = testSample['trueRating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=310)

x_train_text = X_train
x_test_text = X_test

x_train_tokens = tokenizer.texts_to_sequences(x_train_text)
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

In [14]:
len(y_train), len(y_test), len(x_train_text), len(x_test_text)

(100359, 33453, 100359, 33453)

In [15]:
#x_train_tokens[1], x_train_text[1]

In [16]:
import numpy as np

x_train_text[1]
np.array(x_train_tokens[1])
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)


In [17]:
np.mean(num_tokens), np.max(num_tokens)


(111.61465339431442, 4959)

In [18]:
np.std(num_tokens)

133.85901020345165

In [19]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens) 
max_tokens


379

In [20]:
np.sum(num_tokens < max_tokens) / len(num_tokens)


0.9588078797118346

In [21]:
pad = 'pre'
#pad = 'post'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_train_pad.shape, x_test_pad.shape


((100359, 379), (33453, 379))

In [22]:
for i in range(3, 4):
    print(x_train_pad[i])


[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [23]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [24]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [25]:
x_train_text[1]

'very handy and easy to use, these arrive on time and in good condition, nice to have in shop when you need it you need it'

In [26]:
tokens_to_string(x_train_tokens[1])

'very handy and easy to use these arrive on time and in good condition nice to have in shop when you need it you need it'

In [27]:
import tensorflow as tf
tf.__version__

'1.13.2'

In [28]:
#abstract CNN
import modelCNNImplementation as modelCNN
import importlib
importlib.reload(modelCNN)

Using TensorFlow backend.


<module 'modelCNNImplementation' from 'C:\\Study\\Projects\\NLP\\modelCNNImplementation.py'>

In [29]:
model_cnn = modelCNN.compileCNNModel(max_tokens=max_tokens)
model_cnn.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 379, 200)          3000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 378, 100)          40100     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 3,066,213
Trainable params: 3,066,213
Non-trainable params: 0
_________________________________________________________________


In [30]:
import modelRNNImplementation as modelRNN
import importlib
importlib.reload(modelRNN)

<module 'modelRNNImplementation' from 'C:\\Study\\Projects\\NLP\\modelRNNImplementation.py'>

In [31]:
model_rnn = modelRNN.compileRNNModel(max_tokens=max_tokens, embedding_size=200)
model_rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 379, 200)          3000000   
_________________________________________________________________
gru_1 (GRU)                  (None, 379, 16)           10416     
_________________________________________________________________
gru_2 (GRU)                  (None, 379, 8)            600       
_________________________________________________________________
gru_3 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 3,011,177
Trainable params: 3,011,177
Non-trainable params: 0
_________________________________________________________________


In [32]:
index = len(x_train_pad)
index
len(y_train), len(y_test), len(y_train) + len(y_test)

(100359, 33453, 133812)

In [33]:
%%time
model_cnn.fit(x_train_pad, y_train, validation_split=0.05, epochs=3, batch_size=64)
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 95341 samples, validate on 5018 samples
Epoch 1/3
95341/95341 [==============================] - 522s 5ms/step - loss: 0.2291 - acc: 0.9092 - val_loss: 0.1657 - val_acc: 0.9388
Epoch 2/3
95341/95341 [==============================] - 531s 6ms/step - loss: 0.1250 - acc: 0.9529 - val_loss: 0.1364 - val_acc: 0.9426
Epoch 3/3
95341/95341 [==============================] - 396s 4ms/step - loss: 0.0752 - acc: 0.9727 - val_loss: 0.1273 - val_acc: 0.9532
Wall time: 24min 9s


In [34]:
%%time
model_rnn.fit(x_train_pad, y_train, validation_split=0.05, epochs=3, batch_size=128)
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

Train on 95341 samples, validate on 5018 samples
Epoch 1/3
95341/95341 [==============================] - 830s 9ms/step - loss: 0.2848 - acc: 0.8932 - val_loss: 0.1974 - val_acc: 0.9249
Epoch 2/3
95341/95341 [==============================] - 820s 9ms/step - loss: 0.1564 - acc: 0.9439 - val_loss: 0.1549 - val_acc: 0.9422
Epoch 3/3
95341/95341 [==============================] - 841s 9ms/step - loss: 0.1034 - acc: 0.9658 - val_loss: 0.1409 - val_acc: 0.9518
Wall time: 41min 33s


In [35]:
#%%time
cnn_result = model_cnn.evaluate(x_test_pad, y_test)

33453/33453 [==============================] - 45s 1ms/step


In [36]:
#%%time
rnn_result = model_rnn.evaluate(x_test_pad, y_test)

33453/33453 [==============================] - 116s 3ms/step


In [37]:
print("Accuracy: {0:.2%}".format(rnn_result[1]))

Accuracy: 94.87%


In [39]:
print("Accuracy: {0:.2%}".format(cnn_result[1]))


Accuracy: 95.10%


In [40]:
%%time
cnn_y_pred = model_cnn.predict(x=x_test_pad)
cnn_y_pred = cnn_y_pred.T[0]

Wall time: 39.3 s


In [41]:
%%time
rnn_y_pred = model_rnn.predict(x=x_test_pad)
rnn_y_pred = rnn_y_pred.T[0]

Wall time: 1min 47s


In [42]:
pd.DataFrame(cnn_y_pred).describe(), pd.DataFrame(rnn_y_pred).describe()

(                  0
 count  33453.000000
 mean       0.834357
 std        0.332229
 min        0.000071
 25%        0.944408
 50%        0.999085
 75%        0.999959
 max        1.000000,                   0
 count  33453.000000
 mean       0.856336
 std        0.310601
 min        0.008763
 25%        0.978606
 50%        0.994482
 75%        0.995702
 max        0.996268)

In [43]:
cnn_y_pred.shape, rnn_y_pred.shape

((33453,), (33453,))

In [44]:
cnn_cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in cnn_y_pred])
rnn_cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in rnn_y_pred])

In [45]:
len(cnn_cls_pred), len(rnn_cls_pred)

(33453, 33453)

In [46]:
cnn_accuracy = (cnn_cls_pred==y_test).mean()
rnn_accuracy = (rnn_cls_pred==y_test).mean()
cnn_accuracy, rnn_accuracy

(0.9509759961737363, 0.9486742594087226)

In [47]:
from sklearn.metrics import confusion_matrix

In [48]:
#Check for padding
tn, fp, fn, tp = confusion_matrix(y_test, cnn_cls_pred).ravel()
tn, fp, fn, tp

(4431, 653, 987, 27382)

In [49]:
tn, fp, fn, tp = confusion_matrix(y_test, rnn_cls_pred).ravel()
tn, fp, fn, tp

(4033, 1051, 666, 27703)

In [50]:
len(np.where(y_test==0)[0]),len(np.where(y_test==1)[0])

(5084, 28369)

In [51]:
len(np.where(y_train==0)[0]),len(np.where(y_train==1)[0])

(15126, 85233)

In [52]:
cls_true = np.array(y_test)

In [53]:
y_test.index
y_test.values
result = pd.DataFrame()
result['Index'] = y_test.index
result['TrueRating'] = y_test.values
result['CNNRating'] = cnn_cls_pred
result['RNNRating'] = rnn_cls_pred

In [54]:
result.shape

(33453, 4)

In [55]:
# need to drop the duplicates due to negative padding
result.drop_duplicates(subset="Index", inplace=True)
result.shape

(32836, 4)

In [56]:
result.head(1)

,Index,TrueRating,CNNRating,RNNRating
0,96807,1.0,1.0,1.0


In [57]:
df = testSample.loc[result.Index.to_list(), :]
df = df.reset_index()
#A.drop_duplicates()
df= df.drop_duplicates(subset="index")
df.rename(columns={"index": "Index"}, inplace=True)
df.shape, df.columns

((32836, 8),
 Index(['Index', 'masterIndex', 'asin', 'reviewText', 'helpful', 'overall',
        'trueRating', 'CleanedJoin'],
       dtype='object'))

In [58]:
df.head(4)

,Index,masterIndex,asin,reviewText,helpful,overall,trueRating,CleanedJoin
0,96807,105069,B004TEIMVC,"If you're an advanced user, whether that be a ...","[3, 4]",5.0,1.0,if you be an advance user whether that be a ho...
1,120279,130698,B00DQ3F0YY,You really get your money worth with this flas...,"[0, 0]",5.0,1.0,you really get your money worth with this flas...
2,117496,127682,B00BB2MMNE,Simple and clear instructions; including wiri...,"[0, 0]",5.0,1.0,simple and clear instruction include wire the ...
3,43612,47267,B000LG7G0A,The shade was easy to cut and easy to install....,"[0, 0]",5.0,1.0,"the shade be easy to cut and easy to install, ..."


In [59]:
modelResult = pd.merge(df, result, on="Index", how="inner")
modelResult.head(3)

,Index,masterIndex,asin,reviewText,helpful,overall,trueRating,CleanedJoin,TrueRating,CNNRating,RNNRating
0,96807,105069,B004TEIMVC,"If you're an advanced user, whether that be a ...","[3, 4]",5.0,1.0,if you be an advance user whether that be a ho...,1.0,1.0,1.0
1,120279,130698,B00DQ3F0YY,You really get your money worth with this flas...,"[0, 0]",5.0,1.0,you really get your money worth with this flas...,1.0,1.0,1.0
2,117496,127682,B00BB2MMNE,Simple and clear instructions; including wiri...,"[0, 0]",5.0,1.0,simple and clear instruction include wire the ...,1.0,1.0,1.0


In [60]:
import TextBlobImplementation as TextBlob
import importlib
importlib.reload(TextBlob)

<module 'TextBlobImplementation' from 'C:\\Study\\Projects\\NLP\\TextBlobImplementation.py'>

In [61]:
polaritySample = modelResult.iloc[3, :]['CleanedJoin']

TextBlob.getPolarityFromText(polaritySample.split(','))

1.0

In [62]:
import TextPreProcessing as TextProcessing
import importlib
importlib.reload(TextProcessing)

<module 'TextPreProcessing' from 'C:\\Study\\Projects\\NLP\\TextPreProcessing.py'>

In [63]:
%%time
modelResult['simplifiedText'] = modelResult.reviewText.apply(TextProcessing.simplifiedTextParagraph)

Wall time: 19.4 s


In [64]:
%%time
#not very accurate using stemming
#modelResult['textBlobRating'] = modelResult.CleanedJoin.str.split(',')
modelResult['textBlobRating'] = modelResult.simplifiedText.apply(TextBlob.getPolarityFromText)

Wall time: 2min 6s


In [65]:
modelResult.head(3)

,Index,masterIndex,asin,reviewText,helpful,overall,trueRating,CleanedJoin,TrueRating,CNNRating,RNNRating,simplifiedText,textBlobRating
0,96807,105069,B004TEIMVC,"If you're an advanced user, whether that be a ...","[3, 4]",5.0,1.0,if you be an advance user whether that be a ho...,1.0,1.0,1.0,[if you are an advanced user whether that be ...,1.0
1,120279,130698,B00DQ3F0YY,You really get your money worth with this flas...,"[0, 0]",5.0,1.0,you really get your money worth with this flas...,1.0,1.0,1.0,[you really get your money worth with this fla...,1.0
2,117496,127682,B00BB2MMNE,Simple and clear instructions; including wiri...,"[0, 0]",5.0,1.0,simple and clear instruction include wire the ...,1.0,1.0,1.0,[simple and clear instructions including wir...,1.0


In [66]:
from sklearn.metrics import confusion_matrix

y_true = modelResult.TrueRating.to_list()
cnn_pred = modelResult.CNNRating.to_list()
textBlob_pred = modelResult.textBlobRating.to_list()

In [67]:
from sklearn.metrics import confusion_matrix

y_true = modelResult.TrueRating.to_list()
rnn_pred = modelResult.RNNRating.to_list() 
textBlob_pred = modelResult.textBlobRating.to_list()

In [70]:
#report = {'CNN': [], 'RNN': None, 'TextBlob':None}
report = dict()

In [71]:
#tn, fp, fn, tp = confusion_matrix(y_true, cnn_pred).ravel()
report['CNN'] = confusion_matrix(y_true, cnn_pred).ravel()
#tn, fp, fn, tp
report['CNN']

array([ 4024,   443,   987, 27382], dtype=int64)

In [72]:
#tn, fp, fn, tp = confusion_matrix(y_true, rnn_pred).ravel()
report['RNN'] = confusion_matrix(y_true, rnn_pred).ravel()
#tn, fp, fn, tp
report['RNN']

array([ 3654,   813,   666, 27703], dtype=int64)

In [73]:
#tn, fp, fn, tp = confusion_matrix(y_true, textBlob_pred).ravel()
report['TextBlob'] = confusion_matrix(y_true, textBlob_pred).ravel()
#tn, fp, fn, tp

In [ ]:
pd.options.display.max_colwidth = 50

#productsReviewData.cleanedText.head(2)

In [74]:
y_true = np.array(y_true)

In [75]:
reportCard = pd.DataFrame(report).T
reportCard.columns = ['tn', 'fp', 'fn', 'tp']

In [77]:
reportCard.loc['ActualRating'] = [len(np.where(y_true==0)[0]), 0, 0, len(np.where(y_true==1)[0])]
reportCard

,tn,fp,fn,tp
CNN,4024,443,987,27382
RNN,3654,813,666,27703
TextBlob,1336,3131,2026,26343
ActualRating,4467,0,0,28369


In [78]:
reportCard.to_csv("2nd_PaddingNegativeReviews_RNN_CNN_TextBlob.csv")

## 133K review comparison

## padding extra negative reviews

##### we pick up two unique negative extras with the sampling when we padded

tn, fp, fn, tp

(3913, 554, 742, 27627)  CNN for negative reviews padding 
(3445, 1022, 923, 27446) RNN with negative reviews padding
(1142, 3325, 1782, 26587) textBlob with negative reviews padding 

Note: RNN 3*8 word vector with  batch size = 64

## no padding extra negative reviews

tn, fp, fn, tp

(1667, 916, 813, 27531)  CNN without negative reviews padding
(1034, 1549, 270, 28074) RNN without negative reviews padding
(671, 1912, 1767, 26577) textBlob without negative reviews padding


## Multinomial Naive Bayes no padding extra negative reviews

tn, fp, fn, tp

(509, 2074, 4175, 24169) Multinomial Naive Bayes # n-gram (1,2), num_words = 20000
(26, 83, 393, 1871) Multinomial Naive Bayes # n-gram (1,1), num_words = 3000




## 10K review comparison

## padding extra negative reviews

##### we pick up two unique negative extras with the sampling when we padded

tn, fp, fn, tp

(174, 27, 44, 2214) CNN for negative reviews padding 
(113, 88, 34, 2224)  RNN with negative review padding
(50, 151, 112, 2146) textBlob with negative padding 

## no padding extra negative reviews

tn, fp, fn, tp

(31, 78, 37, 2227)  CNN without negative reviews padding
(3, 106, 2, 2262)   RNN without negative reviews padding
(27, 82, 106, 2158) textBlob without negative reviews padding


## Multinomial Naive Bayes no padding extra negative reviews

tn, fp, fn, tp

(12, 97, 232, 2032) Multinomial Naive Bayes # n-gram (1,2), num_words = 10000
(26, 83, 393, 1871) Multinomial Naive Bayes # n-gram (1,1), num_words = 3000


